# Elastic Net回帰の実装 (NumPy)

このノートブックでは、NumPyのみを使用してElastic Net回帰をスクラッチから実装します。
Elastic Netは、L1正則化（Lassoペナルティ）とL2正則化（Ridgeペナルティ）を組み合わせた手法で、Lassoの変数選択能力とRidgeの多重共線性への対処能力を両立させることを目指します。

## 1. ライブラリのインポート

In [1]:
import numpy as np

## 2. Elastic Net回帰の理論的背景

Elastic Netの目的関数は、残差平方和 (RSS) にL1ペナルティとL2ペナルティの両方を加えたものです。
$$ L_{ElasticNet}(\beta_0, \mathbf{\beta}_{rest}) = \frac{1}{2N} ||\mathbf{y} - (\mathbf{X}_{rest}\mathbf{\beta}_{rest} + \beta_0)||^2_2 + \alpha \left( \rho ||\mathbf{\beta}_{rest}||_1 + \frac{1-\rho}{2} ||\mathbf{\beta}_{rest}||_2^2 \right) $$
または、単純化して以下のように書かれることもあります（$1/2N$ の係数の有無や$\alpha$のスケールに注意）。
$$ L_{ElasticNet}(\beta_0, \mathbf{\beta}_{rest}) = \sum_{i=1}^{N} (y_i - (\beta_0 + \sum_{j=1}^{p} x_{ij}\beta_j))^2 + \lambda_1 \sum_{j=1}^{p} |\beta_j| + \lambda_2 \sum_{j=1}^{p} \beta_j^2 $$
scikit-learnのドキュメントに近い形で、$\alpha$ を全体のペナルティの強さ、$\rho$ (または `l1_ratio`) をL1ペナルティとL2ペナルティの混合比率として表現します。
ここでは、実装の都合上、以下の目的関数を考えます（係数 $1/2$ をRSS項の前に置くのが一般的）。
$$ L_{ElasticNet}(\beta_0, \mathbf{\beta}_{rest}) = \frac{1}{2} \sum_{i=1}^{N} (y_i - (\beta_0 + \sum_{j=1}^{p} x_{ij}\beta_j))^2 + \alpha \rho \sum_{j=1}^{p} |\beta_j| + \alpha \frac{1-\rho}{2} \sum_{j=1}^{p} \beta_j^2 $$

ここで、
*   $N$ はサンプル数です。
*   $\beta_0$ は切片項です。
*   $\mathbf{\beta}_{rest} = (\beta_1, ..., \beta_p)^T$ は切片以外の回帰係数のベクトルです。
*   $\mathbf{X}_{rest}$ は切片項に対応する列を除いた特徴量行列です。
*   $\alpha > 0$ は全体の正則化の強さを制御するパラメータです。
*   $0 \le \rho \le 1$ (l1\_ratio) はL1ペナルティとL2ペナルティの混合比率を制御するパラメータです。
    *   $\rho = 1$ のとき、Elastic NetはLasso回帰と一致します。
    *   $\rho = 0$ のとき、Elastic NetはRidge回帰と一致します（ただし、目的関数のL2項の係数が $\alpha/2$ となる）。
*   切片 $\beta_0$ は通常、正則化の対象外とします。

### 最適化: 座標降下法 (Coordinate Descent)
Lassoと同様に、L1ノルムの項があるため、座標降下法を用います。
$\beta_k$ ($k \in \{1, ..., p\}$) を更新する際、他の係数を固定した目的関数を最小化します。
更新式は、ソフトしきい値関数 $S$ を用いて以下のように表されます。

$$ \rho_k = \sum_{i=1}^{N} x_{ik} \left( y_i - \beta_0 - \sum_{j \neq k} x_{ij}\beta_j \right) $$
$$ \beta_k \leftarrow \frac{S(\rho_k, \alpha \rho)}{\sum_{i=1}^{N} x_{ik}^2 + \alpha (1-\rho)} $$

ここで、ソフトしきい値関数 $S(z, \lambda)$ は次のように定義されます。
$$ S(z, \lambda) = \begin{cases} z - \lambda & \text{if } z > 0 \text{ and } \lambda < |z| \\ z + \lambda & \text{if } z < 0 \text{ and } \lambda < |z| \\ 0 & \text{if } \lambda \ge |z| \end{cases} $$
分母の $\sum_{i=1}^{N} x_{ik}^2$ は、特徴量が標準化されていれば $N$ (または $1$) に近くなります。
Lassoの更新式と比較すると、ソフトしきい値の第2引数が $\alpha \rho$ になり、分母に $\alpha (1-\rho)$ が加わっています。これはL2ペナルティの効果です。

切片 $\beta_0$ は、各イテレーションの最後に次のように更新します。
$$ \beta_0 \leftarrow \frac{1}{N} \sum_{i=1}^{N} (y_i - \sum_{j=1}^{p} x_{ij}\beta_j) $$

### $\alpha$ と $\rho$ の効果
*   $\alpha$: 全体的なペナルティの強さ。大きいほど係数は0に近づく。
*   $\rho$: L1とL2の混合比。
    *   $\rho=1$: Lasso回帰。変数選択性が強い。
    *   $\rho=0$: Ridge回帰（に近い形）。係数を全体的に縮小。多重共線性に強い。
    *   $0 < \rho < 1$: LassoとRidgeの特性を併せ持つ。相関の高い変数群がある場合、Lassoが一つだけを選択するのに対し、Elastic Netはグループ全体を選択する傾向がある（グルーピング効果）。
適切な $\alpha$ と $\rho$ の値は、交差検証などを用いて選択する必要があります。

## 3. データセットの準備

簡単なサンプルデータを作成してテストします。

In [2]:
# サンプルデータ
X_train_sample = np.array([
    [1, 1, 0.9], # 特徴量1と3を似せる
    [1, 2, 1.1],
    [2, 2, 1.9],
    [2, 3, 2.2],
    [3, 2, 2.8],
    [3, 4, 3.2],
    [4, 4, 3.9],
    [4, 5, 4.1]
])
np.random.seed(42)
y_train_sample = (2 * X_train_sample[:, 0] + 
                  0.5 * X_train_sample[:, 1] + 
                  1.5 * X_train_sample[:, 2] + # 特徴量3にも係数を持たせる
                  np.random.normal(0, 0.5, X_train_sample.shape[0]))

print("サンプル特徴量 X_train_sample (shape):", X_train_sample.shape)
print(X_train_sample)
print("\nサンプル目的変数 y_train_sample (shape):", y_train_sample.shape)
print(y_train_sample)

サンプル特徴量 X_train_sample (shape): (8, 3)
[[1.  1.  0.9]
 [1.  2.  1.1]
 [2.  2.  1.9]
 [2.  3.  2.2]
 [3.  2.  2.8]
 [3.  4.  3.2]
 [4.  4.  3.9]
 [4.  5.  4.1]]

サンプル目的変数 y_train_sample (shape): (8,)
[ 4.09835708  4.58086785  8.17384427  9.56151493 11.08292331 12.68293152
 16.63960641 17.03371736]


## 4. Elastic Net回帰モデルクラスの実装

### 4.1 `ElasticNetRegression` クラスの実装

In [3]:
class ElasticNetRegression:
    def __init__(self, alpha=1.0, rho=0.5, n_iterations=1000, tol=1e-4):
        self.alpha = alpha              # 全体の正則化パラメータ
        self.rho = rho                  # L1混合比 (l1_ratio)
        self.n_iterations = n_iterations # 座標降下法の最大反復回数
        self.tol = tol                  # 収束判定のための許容誤差
        self.intercept_ = None          # 切片 (β₀)
        self.coef_ = None               # 回帰係数 (β₁, ..., βₚ)

    def _soft_thresholding(self, z, lambda_val):
        if z < -lambda_val:
            return z + lambda_val
        elif z > lambda_val:
            return z - lambda_val
        else:
            return 0

    def fit(self, X, y):
        '''
        訓練データを用いてモデルを学習する
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数). 事前に標準化されていることが望ましい。
            y(ndarray): 目的変数ベクトル (サンプル数,)
        '''
        n_samples, n_features = X.shape

        # 係数の初期化
        self.coef_ = np.zeros(n_features)
        self.intercept_ = np.mean(y) # 初期値をyの平均とする

        # L1ペナルティとL2ペナルティの係数
        l1_penalty = self.alpha * self.rho
        l2_penalty_factor = self.alpha * (1 - self.rho) #分母に加えるのは alpha*(1-rho)

        # 座標降下法
        for iteration in range(self.n_iterations):
            coef_old = np.copy(self.coef_)

            # 切片の更新
            self.intercept_ = np.mean(y - X @ self.coef_)

            # 各特徴量の係数を更新
            for j in range(n_features):
                # ρ_j (rho_k in theory section) の計算
                current_prediction = self.intercept_ + X @ self.coef_
                # β_j * x_j の効果を除いた残差
                residual_for_j = y - (current_prediction - X[:, j] * self.coef_[j]) 
                rho_j_val = X[:, j] @ residual_for_j
                
                # 分母の計算
                sum_sq_xj = np.sum(X[:, j]**2)
                
                denominator = sum_sq_xj + l2_penalty_factor # Lassoとの違い：L2ペナルティ項
                if denominator == 0: 
                    self.coef_[j] = 0
                else:
                    # ソフトしきい値関数を適用
                    self.coef_[j] = self._soft_thresholding(rho_j_val, l1_penalty) / denominator
            
            # 収束判定
            if np.sum(np.abs(self.coef_ - coef_old)) < self.tol:
                # print(f"Converged at iteration {iteration + 1}")
                break
        # if iteration == self.n_iterations - 1:
            # print("Warning: ElasticNet did not converge within max_iterations.")


    def predict(self, X):
        '''
        学習済みモデルを用いて予測を行う
        Parameters:
            X(ndarray): 特徴量行列 (サンプル数, 特徴量数)
        Returns:
            ndarray: 予測値ベクトル (サンプル数,)
        '''
        if self.intercept_ is None or self.coef_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' before 'predict'.")
        
        return X @ self.coef_ + self.intercept_

## 5. モデルの学習と予測 (サンプルデータ)

異なる `alpha` と `rho` の値でElastic Netモデルを学習させ、係数の変化を確認します。
実践では標準化が重要ですが、この小さなサンプルでは標準化せずに行います。

In [4]:
# サンプルデータでテスト (標準化なし)
alphas_sample = [0.1, 0.5, 1.0]
rhos_sample = [0.1, 0.5, 0.9, 1.0] # rho=1.0はLasso

print("サンプルデータでのElastic Netモデルの学習結果 (Xは非標準化):\n")
for alpha_val in alphas_sample:
    for rho_val in rhos_sample:
        if rho_val == 0 and alpha_val == 0: # Ridgeでalpha=0は特異行列問題の可能性
            print(f"Skipping Alpha = {alpha_val}, Rho = {rho_val} (Potential singularity for Ridge with alpha=0)")
            continue
        if rho_val == 0 and alpha_val > 0 : # 今回の実装はrho=0を直接扱えない (l1_penaltyが0になりソフトしきい値の挙動)
                                         # Ridgeは別途実装するか、scikit-learnのElasticNetでrhoを非常に小さくする
            print(f"Skipping Rho = {rho_val} (Pure Ridge behavior not directly handled by this soft-thresholding based ElasticNet, use RidgeRegression class)")
            continue


        model_enet_sample = ElasticNetRegression(alpha=alpha_val, rho=rho_val, n_iterations=3000, tol=1e-5)
        model_enet_sample.fit(X_train_sample, y_train_sample)
        
        print(f"Alpha = {alpha_val}, Rho (l1_ratio) = {rho_val}")
        print(f"  切片 (β₀): {model_enet_sample.intercept_:.4f}")
        print(f"  回帰係数: {np.round(model_enet_sample.coef_, 4)}")
        print("---")

# alpha=0.5, rho=0.5 のモデルで予測
model_enet_sample_pred = ElasticNetRegression(alpha=0.5, rho=0.5, n_iterations=3000, tol=1e-5)
model_enet_sample_pred.fit(X_train_sample, y_train_sample)
y_pred_sample_enet = model_enet_sample_pred.predict(X_train_sample)

print("\n訓練データに対する予測結果 (Alpha=0.5, Rho=0.5):\n")
for i in range(len(y_train_sample)):
    print(f"  実測値: {y_train_sample[i]:.2f}, 予測値: {y_pred_sample_enet[i]:.2f}, 誤差: {y_train_sample[i] - y_pred_sample_enet[i]:.2f}")

サンプルデータでのElastic Netモデルの学習結果 (Xは非標準化):

Alpha = 0.1, Rho (l1_ratio) = 0.1
  切片 (β₀): 0.0562
  回帰係数: [2.0343 0.6236 1.4117]
---
Alpha = 0.1, Rho (l1_ratio) = 0.5
  切片 (β₀): 0.0440
  回帰係数: [2.1568 0.6369 1.2794]
---
Alpha = 0.1, Rho (l1_ratio) = 0.9
  切片 (β₀): -0.0139
  回帰係数: [2.7662 0.768  0.5461]
---
Alpha = 0.1, Rho (l1_ratio) = 1.0
  切片 (β₀): -0.0552
  回帰係数: [3.2138 0.8704 0.    ]
---
Alpha = 0.5, Rho (l1_ratio) = 0.1
  切片 (β₀): 0.1867
  回帰係数: [1.8002 0.6911 1.5155]
---
Alpha = 0.5, Rho (l1_ratio) = 0.5
  切片 (β₀): 0.1744
  回帰係数: [1.8681 0.6446 1.506 ]
---
Alpha = 0.5, Rho (l1_ratio) = 0.9
  切片 (β₀): 0.1468
  回帰係数: [2.1179 0.6303 1.2849]
---
Alpha = 0.5, Rho (l1_ratio) = 1.0
  切片 (β₀): 0.0474
  回帰係数: [3.1787 0.8652 0.    ]
---
Alpha = 1.0, Rho (l1_ratio) = 0.1
  切片 (β₀): 0.3362
  回帰係数: [1.6851 0.7724 1.4774]
---
Alpha = 1.0, Rho (l1_ratio) = 0.5
  切片 (β₀): 0.3145
  回帰係数: [1.761  0.6965 1.4974]
---
Alpha = 1.0, Rho (l1_ratio) = 0.9
  切片 (β₀): 0.2871
  回帰係数: [1.9486 0.6124 1.4179]
---
A

## 6. より実践的なデータセットでの利用と評価 (Diabetes Dataset)

scikit-learnのDiabetesデータセットを使用して、モデルの性能を評価します。
特徴量の標準化が特に重要です。

In [5]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

diabetes = load_diabetes()
X_diabetes, y_diabetes = diabetes.data, diabetes.target

# 特徴量の標準化
scaler_diabetes = StandardScaler()
X_diabetes_scaled = scaler_diabetes.fit_transform(X_diabetes)

# 訓練データとテストデータに分割
X_diabetes_train, X_diabetes_test, y_diabetes_train, y_diabetes_test = train_test_split(
    X_diabetes_scaled, y_diabetes, test_size=0.2, random_state=42
)

# Diabetesデータセットで評価
alphas_practical_enet = [0.01, 0.1, 0.5] # scikit-learnのalphaとはスケールが異なる可能性
rhos_practical_enet = [0.1, 0.5, 0.9]    # l1_ratio

print("\n実践的なデータセット (Diabetes, Xは標準化済み) での評価:\n")
for alpha_val in alphas_practical_enet:
    for rho_val in rhos_practical_enet:
        if rho_val == 0: # 純粋なRidgeは別途
            print(f"Skipping Rho = {rho_val}")
            continue

        model_practical_enet = ElasticNetRegression(alpha=alpha_val, rho=rho_val, n_iterations=5000, tol=1e-6) # イテレーション数とtolを調整
        model_practical_enet.fit(X_diabetes_train, y_diabetes_train)

        y_pred_diabetes_test = model_practical_enet.predict(X_diabetes_test)

        mse_diabetes = mean_squared_error(y_diabetes_test, y_pred_diabetes_test)
        r2_diabetes = r2_score(y_diabetes_test, y_pred_diabetes_test)

        print(f"Alpha = {alpha_val}, Rho (l1_ratio) = {rho_val}")
        print(f"  切片: {model_practical_enet.intercept_:.4f}")
        print(f"  係数の非ゼロ要素数: {np.sum(np.abs(model_practical_enet.coef_) > 1e-6)} / {len(model_practical_enet.coef_)}")
        # print(f"  係数: {np.round(model_practical_enet.coef_, 4)}")
        print(f"  平均二乗誤差 (MSE): {mse_diabetes:.4f}")
        print(f"  決定係数 (R²): {r2_diabetes:.4f}")
        print("---")


実践的なデータセット (Diabetes, Xは標準化済み) での評価:

Alpha = 0.01, Rho (l1_ratio) = 0.1
  切片: 151.3455
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2900.0867
  決定係数 (R²): 0.4526
---
Alpha = 0.01, Rho (l1_ratio) = 0.5
  切片: 151.3456
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2900.1318
  決定係数 (R²): 0.4526
---
Alpha = 0.01, Rho (l1_ratio) = 0.9
  切片: 151.3456
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2900.1770
  決定係数 (R²): 0.4526
---
Alpha = 0.1, Rho (l1_ratio) = 0.1
  切片: 151.3448
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2899.1622
  決定係数 (R²): 0.4528
---
Alpha = 0.1, Rho (l1_ratio) = 0.5
  切片: 151.3451
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2899.5878
  決定係数 (R²): 0.4527
---
Alpha = 0.1, Rho (l1_ratio) = 0.9
  切片: 151.3455
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2900.0285
  決定係数 (R²): 0.4526
---
Alpha = 0.5, Rho (l1_ratio) = 0.1
  切片: 151.3419
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2895.7339
  決定係数 (R²): 0.4534
---
Alpha = 0.5, Rho (l1_ratio) = 0.5
  切片: 151.3433
  係数の非ゼロ要素数: 10 / 10
  平均二乗誤差 (MSE): 2897.4102
  決定係数 (R²): 0.4531
--

## 7. 考察

*   このスクラッチ実装では、Elastic Net回帰の係数推定に座標降下法とElastic Netに対応するソフトしきい値の更新式を用いました。L1ペナルティとL2ペナルティを組み合わせることで、Lassoの変数選択能力とRidgeの安定性を両立させることを目指します。
*   **長所**:
    *   **変数選択とグルーピング効果**: Lassoのように変数を選択しつつ、Ridgeのように相関の高い特徴量群をまとめて扱う（係数を0にするか、グループとして残すか）傾向があります。これは「グルーピング効果」と呼ばれます。
    *   **安定性**: $p > N$ の場合や多重共線性が強い場合に、Lassoよりも安定した解を与えることがあります。
    *   **LassoとRidgeの一般化**: $\rho=1$ でLasso、$\rho \approx 0$ でRidge（の挙動に近い形）となり、両手法の一般化と見なせます。

*   **短所・注意点**:
    *   **最適化アルゴリズム**: Lassoと同様に反復解法が必要です。`n_iterations` や `tol` の設定が収束に影響します。
    *   **ハイパーパラメータの選択**: $\alpha$ と $\rho$ の2つのハイパーパラメータを調整する必要があり、探索空間が広がります。交差検証による適切な選択がより重要になります。
    *   **特徴量のスケーリング**: L1ペナルティとL2ペナルティの両方が係数の大きさに依存するため、学習前に特徴量を標準化することが強く推奨されます。
    *   **$\rho=0$ (純粋なRidge) の扱い**: 今回の座標降下法の実装では、$\rho=0$ の場合、L1ペナルティ項が消え、ソフトしきい値関数が単純な除算になりますが、これはRidge回帰の正規方程式による解と完全に一致するわけではありません（特に$\alpha (1-\rho)$ の項が分母にだけ入るため）。純粋なRidge回帰を座標降下法で解く場合は更新式が異なります。scikit-learnのElasticNetでは $\rho=0$ でRidgeと等価になります。

*   **scikit-learnとの比較**:
    *   scikit-learnの `sklearn.linear_model.ElasticNet` は、効率的なソルバーを実装しており、ハイパーパラメータの調整も容易です。
    *   `alpha` と `l1_ratio` (本実装の `rho`) の定義や目的関数の正規化係数が異なる場合があるため、パラメータを直接比較する際には注意が必要です。
    *   `ElasticNetCV` のように交差検証によるハイパーパラメータの自動選択機能も備えています。本スクラッチ実装は、アルゴリズムの基本的な流れを理解するためのものです。